In [ ]:
from typing import List, Optional, Any, Callable, Tuple, Dict, Set
from dataclasses import dataclass, field
from fractions import Fraction
import math
import time
from collections import deque
import itertools
import random

# 24点游戏目标值
TARGET = 24
# 浮点数比较精度
EPSILON = 1e-9
# 支持的运算符
OPERATORS = ['+', '-', '*', '/']

@dataclass
class ThoughtNode:
    """思维树节点"""
    numbers: List[Fraction]  # 当前剩余数字（使用分数避免精度问题）
    expression: str          # 当前表达式
    value: Fraction          # 当前表达式值
    depth: int               # 节点深度
    children: List['ThoughtNode'] = field(default_factory=list)  # 子节点
    parent: Optional['ThoughtNode'] = None  # 父节点
    
    def __str__(self):
        return f"State: {self.numbers}, Expr: {self.expression}, Value: {float(self.value)}"
    
    def __hash__(self):
        """哈希函数，用于去重"""
        # 对数字排序后转为元组，确保相同数字组合的哈希值相同
        sorted_nums = tuple(sorted(self.numbers))
        return hash((sorted_nums, self.expression))
    
    def __eq__(self, other):
        """相等判断"""
        if not isinstance(other, ThoughtNode):
            return False
        return (sorted(self.numbers) == sorted(other.numbers) and 
                self.expression == other.expression)


class TreeOfThoughts:
    """Tree of Thoughts 实现"""
    
    def __init__(
        self,
        thought_generator: Callable[[ThoughtNode], List[ThoughtNode]],
        state_evaluator: Callable[[ThoughtNode], float],
        goal_checker: Callable[[ThoughtNode], bool],
        pruning_function: Optional[Callable[[ThoughtNode], bool]] = None,
        max_depth: int = 10
    ):
        """
        初始化ToT框架
        
        Args:
            thought_generator: 思维生成器，根据当前节点生成可能的下一步思考
            state_evaluator: 状态评估器，评估节点质量（0-1，越高越好）
            goal_checker: 目标检查器，判断节点是否达到目标
            pruning_function: 剪枝函数，返回True表示需要剪枝
            max_depth: 最大搜索深度
        """
        self.thought_generator = thought_generator
        self.state_evaluator = state_evaluator
        self.goal_checker = goal_checker
        self.pruning_function = pruning_function
        self.max_depth = max_depth
        self.visited_states = set()  # 已访问状态，用于去重
        self.stats = {
            'nodes_expanded': 0,
            'nodes_pruned': 0,
            'max_depth_reached': 0,
            'search_time': 0.0
        }
    
    def reset_stats(self):
        """重置统计信息"""
        self.stats = {
            'nodes_expanded': 0,
            'nodes_pruned': 0,
            'max_depth_reached': 0,
            'search_time': 0.0
        }
        self.visited_states.clear()
    
    def _is_state_visited(self, node: ThoughtNode) -> bool:
        """检查状态是否已访问"""
        return node in self.visited_states
    
    def _mark_state_visited(self, node: ThoughtNode):
        """标记状态为已访问"""
        self.visited_states.add(node)
    
    def bfs_search(self, initial_node: ThoughtNode) -> Optional[ThoughtNode]:
        """
        广度优先搜索
        
        Args:
            initial_node: 初始节点
            
        Returns:
            找到的解节点，或None
        """
        start_time = time.time()
        self.reset_stats()
        
        queue = deque([initial_node])
        self._mark_state_visited(initial_node)
        
        while queue:
            current_node = queue.popleft()
            
            # 更新最大深度
            self.stats['max_depth_reached'] = max(
                self.stats['max_depth_reached'], current_node.depth
            )
            
            # 检查是否达到目标
            if self.goal_checker(current_node):
                self.stats['search_time'] = time.time() - start_time
                return current_node
            
            # 检查深度限制
            if current_node.depth >= self.max_depth:
                continue
            
            # 生成子节点
            child_nodes = self.thought_generator(current_node)
            self.stats['nodes_expanded'] += len(child_nodes)
            
            for child in child_nodes:
                # 检查是否已访问
                if self._is_state_visited(child):
                    continue
                
                # 检查是否需要剪枝
                if self.pruning_function and self.pruning_function(child):
                    self.stats['nodes_pruned'] += 1
                    continue
                
                # 添加到队列
                child.parent = current_node
                current_node.children.append(child)
                self._mark_state_visited(child)
                queue.append(child)
        
        self.stats['search_time'] = time.time() - start_time
        return None
    
    def dfs_search(
        self, 
        initial_node: ThoughtNode,
        depth_limit: Optional[int] = None
    ) -> Optional[ThoughtNode]:
        """
        深度优先搜索
        
        Args:
            initial_node: 初始节点
            depth_limit: 深度限制
            
        Returns:
            找到的解节点，或None
        """
        start_time = time.time()
        self.reset_stats()
        
        if depth_limit is None:
            depth_limit = self.max_depth
        
        # 使用迭代DFS避免递归深度限制
        stack = [(initial_node, 0)]  # (节点, 当前深度)
        self._mark_state_visited(initial_node)
        
        while stack:
            current_node, current_depth = stack.pop()
            
            # 更新最大深度
            self.stats['max_depth_reached'] = max(
                self.stats['max_depth_reached'], current_depth
            )
            
            # 检查是否达到目标
            if self.goal_checker(current_node):
                self.stats['search_time'] = time.time() - start_time
                return current_node
            
            # 检查深度限制
            if current_depth >= depth_limit:
                continue
            
            # 生成子节点
            child_nodes = self.thought_generator(current_node)
            self.stats['nodes_expanded'] += len(child_nodes)
            
            # 对子节点进行评估排序（可选，使搜索更智能）
            evaluated_children = [
                (child, self.state_evaluator(child))
                for child in child_nodes
                if not self._is_state_visited(child)
            ]
            
            # 按评估值降序排序（评估值越高，优先级越高）
            evaluated_children.sort(key=lambda x: x[1], reverse=True)
            
            for child, score in evaluated_children:
                # 检查是否需要剪枝
                if self.pruning_function and self.pruning_function(child):
                    self.stats['nodes_pruned'] += 1
                    continue
                
                # 添加到栈
                child.parent = current_node
                current_node.children.append(child)
                self._mark_state_visited(child)
                stack.append((child, current_depth + 1))
        
        self.stats['search_time'] = time.time() - start_time
        return None
    
    def beam_search(
        self, 
        initial_node: ThoughtNode,
        beam_width: int = 3
    ) -> Optional[ThoughtNode]:
        """
        束搜索（Beam Search）
        
        Args:
            initial_node: 初始节点
            beam_width: 束宽度
            
        Returns:
            找到的解节点，或None
        """
        start_time = time.time()
        self.reset_stats()
        
        # 当前层节点（按评估值排序）
        current_level = [(initial_node, self.state_evaluator(initial_node))]
        self._mark_state_visited(initial_node)
        
        depth = 0
        
        while current_level and depth < self.max_depth:
            next_level = []
            
            for current_node, _ in current_level:
                # 检查是否达到目标
                if self.goal_checker(current_node):
                    self.stats['search_time'] = time.time() - start_time
                    return current_node
                
                # 生成子节点
                child_nodes = self.thought_generator(current_node)
                self.stats['nodes_expanded'] += len(child_nodes)
                
                for child in child_nodes:
                    # 检查是否已访问
                    if self._is_state_visited(child):
                        continue
                    
                    # 检查是否需要剪枝
                    if self.pruning_function and self.pruning_function(child):
                        self.stats['nodes_pruned'] += 1
                        continue
                    
                    # 评估子节点
                    child_score = self.state_evaluator(child)
                    child.parent = current_node
                    current_node.children.append(child)
                    self._mark_state_visited(child)
                    
                    next_level.append((child, child_score))
            
            # 按评估值排序，只保留beam_width个最佳节点
            next_level.sort(key=lambda x: x[1], reverse=True)
            current_level = next_level[:beam_width]
            depth += 1
            
            # 更新最大深度
            self.stats['max_depth_reached'] = max(
                self.stats['max_depth_reached'], depth
            )
        
        self.stats['search_time'] = time.time() - start_time
        return None
    
    def get_solution_path(self, solution_node: ThoughtNode) -> List[ThoughtNode]:
        """
        获取从根节点到解节点的路径
        
        Args:
            solution_node: 解节点
            
        Returns:
            节点路径列表
        """
        path = []
        current = solution_node
        
        while current is not None:
            path.append(current)
            current = current.parent
        
        return list(reversed(path))
    
    def print_stats(self):
        """打印搜索统计信息"""
        print(f"搜索统计:")
        print(f"  扩展节点数: {self.stats['nodes_expanded']}")
        print(f"  剪枝节点数: {self.stats['nodes_pruned']}")
        print(f"  最大搜索深度: {self.stats['max_depth_reached']}")
        print(f"  搜索时间: {self.stats['search_time']:.4f}秒")
        print(f"  已访问状态数: {len(self.visited_states)}")


class Point24Solver:
    """24点求解器"""
    
    def __init__(
        self,
        search_strategy: str = 'bfs',
        max_depth: int = 6,
        use_fractions: bool = True,
        allow_parentheses: bool = True
    ):
        """
        初始化24点求解器
        
        Args:
            search_strategy: 搜索策略 ('bfs', 'dfs', 'beam')
            max_depth: 最大搜索深度
            use_fractions: 是否使用分数运算（避免精度问题）
            allow_parentheses: 是否允许括号
        """
        self.search_strategy = search_strategy
        self.max_depth = max_depth
        self.use_fractions = use_fractions
        self.allow_parentheses = allow_parentheses
        
        # 创建ToT框架
        self.tot = TreeOfThoughts(
            thought_generator=self._generate_thoughts,
            state_evaluator=self._evaluate_state,
            goal_checker=self._check_goal,
            pruning_function=self._prune_state,
            max_depth=max_depth
        )
    
    def _to_number(self, num: Any) -> Fraction:
        """转换为数字（整数或分数）"""
        if isinstance(num, Fraction):
            return num
        elif isinstance(num, (int, float)):
            return Fraction(num).limit_denominator()
        else:
            return Fraction(str(num)).limit_denominator()
    
    def _generate_thoughts(self, node: ThoughtNode) -> List[ThoughtNode]:
        """
        生成下一步思考（状态扩展）
        
        Args:
            node: 当前节点
            
        Returns:
            子节点列表
        """
        children = []
        numbers = node.numbers
        
        # 如果只有一个数字，无法继续运算
        if len(numbers) <= 1:
            return children
        
        # 遍历所有可能的数字对
        n = len(numbers)
        for i in range(n):
            for j in range(i + 1, n):
                a = numbers[i]
                b = numbers[j]
                a_expr = node.expression if len(node.expression) > 0 else str(float(a))
                b_expr = node.expression if len(node.expression) > 0 else str(float(b))
                
                # 尝试所有运算符
                for op in OPERATORS:
                    # 生成新数字和表达式
                    new_num, new_expr, valid = self._apply_operation(a, b, op, a_expr, b_expr)
                    
                    if not valid:
                        continue
                    
                    # 创建剩余数字列表
                    remaining_numbers = []
                    for k in range(n):
                        if k != i and k != j:
                            remaining_numbers.append(numbers[k])
                    
                    # 添加新数字
                    remaining_numbers.append(new_num)
                    
                    # 创建新节点
                    child_node = ThoughtNode(
                        numbers=remaining_numbers,
                        expression=new_expr,
                        value=new_num,
                        depth=node.depth + 1
                    )
                    
                    children.append(child_node)
        
        return children
    
    def _apply_operation(
        self, 
        a: Fraction, 
        b: Fraction, 
        op: str,
        a_expr: str,
        b_expr: str
    ) -> Tuple[Fraction, str, bool]:
        """
        应用运算
        
        Returns:
            (结果值, 表达式, 是否有效)
        """
        # 处理表达式（添加括号如果需要）
        expr_a = a_expr if not self._needs_parentheses(a_expr, op, True) else f"({a_expr})"
        expr_b = b_expr if not self._needs_parentheses(b_expr, op, False) else f"({b_expr})"
        
        try:
            if op == '+':
                result = a + b
                expr = f"{expr_a} + {expr_b}"
            elif op == '-':
                result = a - b
                expr = f"{expr_a} - {expr_b}"
            elif op == '*':
                result = a * b
                expr = f"{expr_a} * {expr_b}"
            elif op == '/':
                # 检查除数是否为0
                if b == 0:
                    return None, None, False
                result = a / b
                expr = f"{expr_a} / {expr_b}"
            else:
                return None, None, False
            
            return result, expr, True
            
        except ZeroDivisionError:
            return None, None, False
    
    def _needs_parentheses(self, expr: str, parent_op: str, is_left: bool) -> bool:
        """
        判断表达式是否需要括号
        
        Args:
            expr: 子表达式
            parent_op: 父运算的运算符
            is_left: 是否为左子表达式
            
        Returns:
            是否需要括号
        """
        if not self.allow_parentheses:
            return False
        
        # 如果表达式是单个数字，不需要括号
        if expr.replace('.', '').replace('-', '').isdigit():
            return False
        
        # 检查表达式的运算符优先级
        if '+' in expr or '-' in expr:
            expr_priority = 1
        elif '*' in expr or '/' in expr:
            expr_priority = 2
        else:
            expr_priority = 3  # 单个数字
        
        # 父运算的优先级
        if parent_op in ('+', '-'):
            parent_priority = 1
        else:  # '*', '/'
            parent_priority = 2
        
        # 需要括号的情况：
        # 1. 子表达式优先级低于父运算
        # 2. 除法中右子表达式是加减法（需要括号）
        # 3. 减法中右子表达式是加减法（需要括号）
        if expr_priority < parent_priority:
            return True
        
        if parent_op == '/' and not is_left and expr_priority == 1:
            return True
        
        if parent_op == '-' and not is_left and expr_priority == 1:
            return True
        
        return False
    
    def _evaluate_state(self, node: ThoughtNode) -> float:
        """
        评估状态质量
        
        Returns:
            评估分数 (0-1，越高越好)
        """
        # 如果只剩一个数字，评估其接近24的程度
        if len(node.numbers) == 1:
            value = float(node.numbers[0])
            diff = abs(value - TARGET)
            
            # 如果正好等于24，返回最高分
            if diff < EPSILON:
                return 1.0
            
            # 使用指数衰减函数评估接近程度
            return math.exp(-diff / 5.0)
        
        # 否则，评估剩余数字的组合可能性
        # 启发式：数字越少越好，数字越接近24越好
        score = 0.0
        
        # 数字数量评分（越少越好）
        num_count_score = (4 - len(node.numbers)) / 3.0
        
        # 数字接近24的程度评分
        closeness_score = 0.0
        for num in node.numbers:
            value = float(num)
            diff = abs(value - TARGET)
            closeness_score += math.exp(-diff / 10.0)
        
        closeness_score /= len(node.numbers)
        
        # 组合可能性评分（检查是否有数字对可以运算得到接近24的结果）
        combination_score = 0.0
        numbers = [float(num) for num in node.numbers]
        
        for i in range(len(numbers)):
            for j in range(i + 1, len(numbers)):
                a, b = numbers[i], numbers[j]
                
                # 尝试所有运算
                for a_op_b in [a + b, a - b, b - a, a * b]:
                    if abs(a_op_b - TARGET) < 5.0:  # 允许一定误差
                        combination_score += 0.1
                
                # 除法（避免除0）
                if b != 0:
                    if abs(a / b - TARGET) < 5.0:
                        combination_score += 0.1
                if a != 0:
                    if abs(b / a - TARGET) < 5.0:
                        combination_score += 0.1
        
        combination_score = min(combination_score, 1.0)
        
        # 综合评分
        score = 0.4 * num_count_score + 0.4 * closeness_score + 0.2 * combination_score
        return min(score, 1.0)
    
    def _check_goal(self, node: ThoughtNode) -> bool:
        """检查是否达到目标（24）"""
        if len(node.numbers) != 1:
            return False
        
        value = float(node.numbers[0])
        return abs(value - TARGET) < EPSILON
    
    def _prune_state(self, node: ThoughtNode) -> bool:
        """
        剪枝函数
        
        Returns:
            True表示需要剪枝
        """
        # 检查是否只剩一个数字但离24太远
        if len(node.numbers) == 1:
            value = float(node.numbers[0])
            diff = abs(value - TARGET)
            
            # 如果差值大于10，剪枝
            if diff > 10.0:
                return True
        
        # 检查是否不可能达到24
        # 启发式：如果所有数字都太小或太大，不可能得到24
        numbers = [float(num) for num in node.numbers]
        max_possible = self._get_max_possible_value(numbers)
        min_possible = self._get_min_possible_value(numbers)
        
        if max_possible < TARGET - EPSILON or min_possible > TARGET + EPSILON:
            return True
        
        return False
    
    def _get_max_possible_value(self, numbers: List[float]) -> float:
        """获取可能的最大值（使用乘法和加法）"""
        if not numbers:
            return 0.0
        
        # 排序，大的数相乘和相加
        sorted_nums = sorted(numbers, reverse=True)
        
        # 尝试多种组合策略
        max_val = sorted_nums[0]
        for i in range(1, len(sorted_nums)):
            # 尝试乘法和加法，取最大值
            max_val = max(max_val * sorted_nums[i], max_val + sorted_nums[i])
        
        return max_val
    
    def _get_min_possible_value(self, numbers: List[float]) -> float:
        """获取可能的最小值（使用减法和除法）"""
        if not numbers:
            return 0.0
        
        # 排序，小的数除和减
        sorted_nums = sorted(numbers)
        
        # 尝试多种组合策略
        min_val = sorted_nums[0]
        for i in range(1, len(sorted_nums)):
            # 尝试除法和减法，取最小值
            if sorted_nums[i] != 0:
                min_val = min(min_val / sorted_nums[i], min_val - sorted_nums[i])
        
        return min_val
    
    def solve(self, numbers: List[int]) -> Optional[str]:
        """
        求解24点
        
        Args:
            numbers: 4个数字（1-13）
            
        Returns:
            表达式字符串，无解返回None
        """
        # 验证输入
        if len(numbers) != 4:
            raise ValueError("必须提供4个数字")
        
        for num in numbers:
            if not (1 <= num <= 13):
                raise ValueError(f"数字必须在1-13之间: {num}")
        
        # 转换为分数
        if self.use_fractions:
            numbers_frac = [self._to_number(num) for num in numbers]
        else:
            numbers_frac = [Fraction(num) for num in numbers]
        
        # 创建初始节点
        initial_node = ThoughtNode(
            numbers=numbers_frac,
            expression="",  # 初始无表达式
            value=Fraction(0),
            depth=0
        )
        
        # 根据策略选择搜索方法
        solution_node = None
        
        if self.search_strategy == 'bfs':
            print("使用广度优先搜索(BFS)...")
            solution_node = self.tot.bfs_search(initial_node)
        elif self.search_strategy == 'dfs':
            print("使用深度优先搜索(DFS)...")
            solution_node = self.tot.dfs_search(initial_node)
        elif self.search_strategy == 'beam':
            print("使用束搜索(Beam Search)...")
            solution_node = self.tot.beam_search(initial_node, beam_width=3)
        else:
            raise ValueError(f"不支持的搜索策略: {self.search_strategy}")
        
        # 打印搜索统计
        self.tot.print_stats()
        
        if solution_node:
            # 获取解决方案路径
            path = self.tot.get_solution_path(solution_node)
            
            # 提取最终表达式
            final_expr = solution_node.expression
            
            # 验证结果
            try:
                # 安全地计算表达式值
                result = self._evaluate_expression(final_expr)
                if abs(result - TARGET) < EPSILON:
                    return f"{final_expr} = 24"
                else:
                    print(f"警告: 表达式计算结果为 {result}, 不是24")
                    return None
            except Exception as e:
                print(f"计算表达式时出错: {e}")
                return None
        else:
            print("未找到解决方案")
            return None
    
    def _evaluate_expression(self, expr: str) -> float:
        """安全地计算表达式值"""
        # 替换分数表示
        expr = expr.replace('/', '/')
        
        try:
            # 使用eval计算，但确保安全
            allowed_chars = set('0123456789+-*/(). ')
            if all(c in allowed_chars for c in expr):
                result = eval(expr)
                return float(result)
            else:
                raise ValueError("表达式包含不安全字符")
        except Exception:
            # 如果eval失败，尝试手动解析
            return self._manual_evaluate(expr)
    
    def _manual_evaluate(self, expr: str) -> float:
        """手动计算表达式值（简化版）"""
        # 移除空格
        expr = expr.replace(' ', '')
        
        # 处理括号
        while '(' in expr:
            start = expr.rfind('(')
            end = expr.find(')', start)
            if end == -1:
                break
            
            sub_expr = expr[start+1:end]
            sub_value = self._evaluate_simple(sub_expr)
            expr = expr[:start] + str(sub_value) + expr[end+1:]
        
        # 计算最终表达式
        return self._evaluate_simple(expr)
    
    def _evaluate_simple(self, expr: str) -> float:
        """计算简单表达式（无括号）"""
        # 分割数字和运算符
        import re
        
        # 匹配数字和运算符
        tokens = re.findall(r'\d+\.?\d*|[+\-*/]', expr)
        
        # 先处理乘除
        i = 0
        while i < len(tokens):
            if tokens[i] in '*/':
                left = float(tokens[i-1])
                right = float(tokens[i+1])
                
                if tokens[i] == '*':
                    result = left * right
                else:  # '/'
                    if right == 0:
                        raise ZeroDivisionError("除零错误")
                    result = left / right
                
                # 替换这三个token为结果
                tokens[i-1:i+2] = [str(result)]
                i -= 1
            else:
                i += 1
        
        # 再处理加减
        result = float(tokens[0])
        i = 1
        while i < len(tokens):
            if tokens[i] == '+':
                result += float(tokens[i+1])
            elif tokens[i] == '-':
                result -= float(tokens[i+1])
            i += 2
        
        return result
    
    def find_all_solutions(self, numbers: List[int], max_solutions: int = 10) -> List[str]:
        """
        查找所有解决方案
        
        Args:
            numbers: 4个数字
            max_solutions: 最大解数量
            
        Returns:
            解决方案列表
        """
        solutions = set()
        
        # 尝试多种数字排列（因为初始表达式可能不同）
        for perm in itertools.permutations(numbers):
            # 创建初始节点
            numbers_frac = [self._to_number(num) for num in perm]
            initial_node = ThoughtNode(
                numbers=numbers_frac,
                expression="",
                value=Fraction(0),
                depth=0
            )
            
            # 使用DFS搜索所有解
            self.tot.reset_stats()
            stack = [initial_node]
            
            while stack and len(solutions) < max_solutions:
                current_node = stack.pop()
                
                if self._check_goal(current_node):
                    # 验证表达式
                    expr = current_node.expression
                    try:
                        result = self._evaluate_expression(expr)
                        if abs(result - TARGET) < EPSILON:
                            solutions.add(f"{expr} = 24")
                    except:
                        pass
                    
                    if len(solutions) >= max_solutions:
                        break
                
                # 生成子节点
                if current_node.depth < self.max_depth:
                    children = self._generate_thoughts(current_node)
                    for child in children:
                        if not self._prune_state(child):
                            child.parent = current_node
                            current_node.children.append(child)
                            stack.append(child)
        
        return list(solutions)


def test_solver():
    """测试求解器"""
    print("=== 24点游戏求解器测试 ===\n")
    
    test_cases = [
        ([3, 3, 8, 8], "8/(3-8/3)"),
        ([1, 2, 3, 4], "1*2*3*4"),
        ([6, 6, 6, 6], "6*6-6-6"),
        ([5, 5, 5, 1], "(5-1/5)*5"),
        ([1, 1, 1, 1], None),  # 无解
        ([2, 3, 4, 5], None),  # 有解
        ([10, 10, 4, 4], None),  # 有解
    ]
    
    # 测试不同搜索策略
    strategies = ['bfs', 'dfs', 'beam']
    
    for strategy in strategies:
        print(f"\n=== 测试 {strategy.upper()} 策略 ===")
        solver = Point24Solver(search_strategy=strategy, max_depth=6)
        
        for numbers, expected in test_cases:
            print(f"\n输入: {numbers}")
            try:
                solution = solver.solve(numbers)
                print(f"结果: {solution}")
                
                if expected is None and solution is None:
                    print("✓ 正确 (无解)")
                elif expected is not None and solution is not None:
                    # 验证结果
                    try:
                        result = eval(solution.split('=')[0].strip())
                        if abs(result - 24) < 0.001:
                            print("✓ 正确 (有解)")
                        else:
                            print(f"✗ 错误: 计算结果为 {result}")
                    except:
                        print("✗ 错误: 无法计算表达式")
                else:
                    print(f"✗ 错误: 预期{'有解' if expected else '无解'}")
            except Exception as e:
                print(f"✗ 错误: {e}")


def interactive_mode():
    """交互模式"""
    print("=== 24点游戏求解器 ===\n")
    print("规则: 给定4个数字(1-13)，使用加减乘除和括号得到24")
    
    while True:
        print("\n" + "=" * 50)
        
        # 获取输入
        try:
            input_str = input("请输入4个数字(用空格分隔，或输入'quit'退出): ").strip()
            
            if input_str.lower() in ['quit', 'exit', 'q']:
                print("感谢使用！")
                break
            
            # 解析数字
            numbers = [int(num) for num in input_str.split()]
            
            if len(numbers) != 4:
                print("错误: 必须输入4个数字")
                continue
            
            # 验证数字范围
            valid = True
            for num in numbers:
                if not (1 <= num <= 13):
                    print(f"错误: 数字 {num} 不在1-13范围内")
                    valid = False
                    break
            
            if not valid:
                continue
            
            # 选择搜索策略
            print("\n请选择搜索策略:")
            print("1. 广度优先搜索(BFS) - 保证找到最短解")
            print("2. 深度优先搜索(DFS) - 可能更快找到解")
            print("3. 束搜索(Beam Search) - 平衡速度和质量")
            
            strategy_choice = input("请选择(1/2/3，默认1): ").strip()
            
            if strategy_choice == '2':
                strategy = 'dfs'
            elif strategy_choice == '3':
                strategy = 'beam'
            else:
                strategy = 'bfs'
            
            # 求解
            print(f"\n使用{strategy.upper()}策略求解: {numbers}")
            
            solver = Point24Solver(search_strategy=strategy, max_depth=7)
            solution = solver.solve(numbers)
            
            if solution:
                print(f"\n✓ 找到解决方案: {solution}")
                
                # 询问是否查找所有解
                find_all = input("\n是否查找所有解? (y/n，默认n): ").strip().lower()
                if find_all == 'y':
                    print("查找所有解中...")
                    all_solutions = solver.find_all_solutions(numbers, max_solutions=20)
                    
                    if all_solutions:
                        print(f"找到 {len(all_solutions)} 个解:")
                        for i, sol in enumerate(all_solutions[:10], 1):
                            print(f"  {i}. {sol}")
                        if len(all_solutions) > 10:
                            print(f"  ... 还有 {len(all_solutions)-10} 个解未显示")
                    else:
                        print("未找到其他解")
            else:
                print("\n✗ 未找到解决方案")
                
                # 验证是否真的无解
                verify = input("是否验证是否真的无解? (y/n，默认n): ").strip().lower()
                if verify == 'y':
                    # 使用更深的搜索
                    print("使用深度搜索验证中...")
                    deep_solver = Point24Solver(search_strategy='dfs', max_depth=10)
                    deep_solution = deep_solver.solve(numbers)
                    
                    if deep_solution:
                        print(f"✓ 深度搜索找到解: {deep_solution}")
                    else:
                        print("✓ 确认无解")
            
        except ValueError as e:
            print(f"输入错误: {e}")
        except KeyboardInterrupt:
            print("\n\n程序被中断")
            break
        except Exception as e:
            print(f"发生错误: {e}")


def batch_test():
    """批量测试"""
    print("=== 批量测试 ===")
    
    # 生成随机测试用例
    import random
    
    test_count = 20
    solvable_count = 0
    total_time = 0.0
    
    strategies = ['bfs', 'dfs']
    
    for strategy in strategies:
        print(f"\n使用 {strategy.upper()} 策略:")
        
        for i in range(test_count):
            # 生成随机数字
            numbers = [random.randint(1, 13) for _ in range(4)]
            
            print(f"\n测试 {i+1}/{test_count}: {numbers}")
            
            solver = Point24Solver(search_strategy=strategy, max_depth=6)
            
            try:
                start_time = time.time()
                solution = solver.solve(numbers)
                elapsed = time.time() - start_time
                
                if solution:
                    print(f"  ✓ 有解 ({elapsed:.3f}秒)")
                    solvable_count += 1
                else:
                    print(f"  ✗ 无解 ({elapsed:.3f}秒)")
                
                total_time += elapsed
            except Exception as e:
                print(f"  ! 错误: {e}")
        
        print(f"\n统计:")
        print(f"  有解率: {solvable_count}/{test_count} ({solvable_count/test_count*100:.1f}%)")
        print(f"  平均时间: {total_time/test_count:.3f}秒")


if __name__ == "__main__":
    print("=== 24点游戏求解器 ===\n")
    print("Tree of Thoughts (ToT) 算法实现")
    print("支持搜索策略: BFS, DFS, Beam Search\n")
    
    print("请选择模式:")
    print("1. 交互模式")
    print("2. 运行测试")
    print("3. 批量测试")
    
    choice = input("请选择(1/2/3，默认1): ").strip()
    
    if choice == '2':
        test_solver()
    elif choice == '3':
        batch_test()
    else:
        interactive_mode()